In [3]:
from google.colab.patches import cv2_imshow

In [7]:
pip install opencv-python

In [9]:
from google.colab.patches import cv2_imshow

In [17]:
from collections import defaultdict
import cv2
import numpy as np
from ultralytics import YOLO

# Load the YOLO11 model
model = YOLO("yolo11m.pt")

# Open the video file
video_path = "/content/PATH-TO-VIDEO.mp4"
cap = cv2.VideoCapture(video_path)

# Get the width and height of the frames
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# Define the codec and create a VideoWriter object to save the output video
output_path = '/content/video_output.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for .mp4 files
out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

# Store the track history
track_history = defaultdict(lambda: [])

# Function to extend tracking lines based on previous points
def extend_tracking_line(track):
    if len(track) < 2:
        return track  # Not enough points to extend

    # Take the last two points to get the direction of movement
    x1, y1 = track[-2]
    x2, y2 = track[-1]

    # Calculate direction vector (simple extrapolation)
    dx = x2 - x1
    dy = y2 - y1

    # Extend the line by 15% in the direction of movement (adjusted from 10%)
    extended_x = x2 + 0.15 * dx
    extended_y = y2 + 0.15 * dy

    # Append the extended point
    extended_track = track.copy()
    extended_track.append((extended_x, extended_y))

    return extended_track

# Loop through the video frames
while cap.isOpened():
    # Read a frame from the video
    success, frame = cap.read()

    if success:
        # Run YOLO11 tracking on the frame, persisting tracks between frames
        results = model.track(frame, persist=True)

        # Get the boxes and track IDs
        boxes = results[0].boxes.xywh.cpu()
        track_ids = results[0].boxes.id.int().cpu().tolist()

        # Visualize the results on the frame
        annotated_frame = results[0].plot()

        # Plot the tracks
        for box, track_id in zip(boxes, track_ids):
            x, y, w, h = box
            track = track_history[track_id]
            track.append((float(x), float(y)))  # x, y center point
            if len(track) > 30:  # retain 30 tracks for 30 frames
                track.pop(0)

            # Extend the tracking line
            extended_track = extend_tracking_line(track)

            # Draw the extended tracking line in yellow color (BGR: 0, 255, 255)
            points = np.hstack(extended_track).astype(np.int32).reshape((-1, 1, 2))
            cv2.polylines(annotated_frame, [points], isClosed=False, color=(0, 255, 255), thickness=10)

        # Write the annotated frame to the output video file
        out.write(annotated_frame)
    else:
        # Break the loop if the end of the video is reached
        break

# Release the video capture and writer objects
cap.release()
out.release()
cv2.destroyAllWindows()

print(f"Video saved at {output_path}")


100%|██████████| 38.8M/38.8M [00:00<00:00, 370MB/s]



0: 384x640 12 persons, 827.8ms
Speed: 3.7ms preprocess, 827.8ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 12 persons, 973.9ms
Speed: 3.6ms preprocess, 973.9ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 14 persons, 1 suitcase, 1583.0ms
Speed: 4.2ms preprocess, 1583.0ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 14 persons, 1 suitcase, 2297.8ms
Speed: 12.0ms preprocess, 2297.8ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 14 persons, 1 suitcase, 1913.3ms
Speed: 4.7ms preprocess, 1913.3ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 14 persons, 1 suitcase, 778.5ms
Speed: 4.1ms preprocess, 778.5ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 16 persons, 1 suitcase, 771.4ms
Speed: 9.4ms preprocess, 771.4ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 16 persons,